In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
from io import StringIO
import time
import warnings
pd.set_option('display.max_columns', None)


In [ ]:
### ----- DRAFT ----- ###
warnings.filterwarnings("ignore")
draft = pd.DataFrame(columns=['Year', 'Rnd', 'Pick', 'Team', 'Player', 'Pos', 'Age', 'To', 'AP1', 'PB', 'St', 'wAV', 'DrAV', 'G', 'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int',
                    'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rec_Att', 'Rec_Yds', 'Rec_TD', 'Def_solo', 'Def_Int', 'Def_Sk', 'College'])

for year in range(1936, datetime.now().year + 1):
    if year % 9 == 0:
        time.sleep(60)
    # --- Collect Data --- #
    s = 'https://www.pro-football-reference.com/years/%s/draft.htm' % str(year)
    url = requests.get(s)
    print(url)
    parse = BeautifulSoup(url.text, "html.parser").prettify()                                               
    start = parse.find("<table")
    end = parse.find("</table")
    temp = pd.read_html(parse[start:end+8])[0]

    # --- Manipulate Data --- #
    temp.columns = temp.columns.droplevel()
    if year < 1994:
        temp.insert(24, 'Def_solo', np.nan)
    temp.insert(0, 'Year', year)
    temp.columns = ['Year', 'Rnd', 'Pick', 'Team', 'Player', 'Pos', 'Age', 'To', 'AP1', 'PB', 'St', 'wAV', 'DrAV', 'G', 'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int',
                    'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rec_Att', 'Rec_Yds', 'Rec_TD', 'Def_solo', 'Def_Int', 'Def_Sk', 'College', 'DELETE']
    temp.drop(['DELETE'], axis = 1, inplace = True)
    temp = temp[temp['Player'] != 'Player']

    # --- Combine Data --- #
    draft = pd.concat([draft, temp])

    print(year)

draft.to_csv('C:\\Users\\pensh\\Desktop\\VSCode\\DataBase\\Data\\Draft\\Draft.csv')
draft

In [5]:
### ----- SUPPLEMENTAL DRAFT ----- ###
warnings.filterwarnings("ignore")

# --- Scrape Data --- #
s = 'https://en.wikipedia.org/wiki/List_of_NFL_supplemental_draft_picks'
url = requests.get(s)
parse = BeautifulSoup(url.text, "html.parser").prettify()
start = [m.start() for m in re.finditer('<table', parse)]
end = [m.start() for m in re.finditer('</table', parse)]
supplemental = pd.read_html(parse[start[3]:end[3]+8])[0]

# --- Manipulate Data --- #
supplemental['Player'] = supplemental['Player'].str.split(pat = ' †')
supplemental['Player'] = supplemental['Player'].apply(lambda x: x[0])
supplemental['Player'] = supplemental['Player'].str.split(pat = r'\[\d+\]')
supplemental['Player'] = supplemental['Player'].apply(lambda x: x[0])

supplemental.to_csv('C:\\Users\\pensh\\Desktop\\VSCode\\DataBase\\Data\\Draft\\Supplemental.csv')

In [ ]:
### ----- Combine ----- ###
warnings.filterwarnings("ignore")
combine = pd.DataFrame(columns=['Year', 'Player', 'Pos', 'College', 'DELETE', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle', 'Drafted'])

for year in range(2000, datetime.now().year + 1):
    if year % 9 == 0:
        time.sleep(60)
    # --- Collect Data --- #
    s = 'https://www.pro-football-reference.com/draft/%s-combine.htm' % str(year)
    url = requests.get(s)
    print(url)
    parse = BeautifulSoup(url.text, "html.parser").prettify()                                               
    start = parse.find("<table")
    end = parse.find("</table")
    temp = pd.read_html(parse[start:end+8])[0]

    # --- Manipulate Data --- #
    temp.insert(0, 'Year', year)
    temp.columns = ['Year', 'Player', 'Pos', 'College', 'DELETE', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle', 'Drafted']
    temp.drop(['DELETE'], axis = 1, inplace = True)
    temp = temp[temp['Player'] != 'Player']

    # --- Combine Data --- #
    combine = pd.concat([combine, temp])

    print(year)

combine.to_csv('C:\\Users\\pensh\\Desktop\\VSCode\\DataBase\\Data\\Combine\\Combine.csv')
combine

In [ ]:
### ----- Transactions ----- ###
warnings.filterwarnings("ignore")
transactions = pd.DataFrame(columns=['Year', 'Month', 'Type', 'From', 'To', 'Date', 'Player', 'Position', 'Transaction'])
type = ['trades', 'signings', 'reserve-list', 'waivers', 'terminations', 'other']

for t in type:
    for year in range(1965, datetime.now().year + 1):
        for month in range(1, 13, 1):
            # --- Skip Future --- #
            if (year == datetime.now().year) & (month > datetime.now().month):
                continue

            # --- Catch Empty Tables --- #
            try:
                # --- Collect Data --- #
                s = 'https://www.nfl.com/transactions/league/%s/%s/%s' % (t, str(year), str(month))
                url = requests.get(s)
                print(url)
                parse = BeautifulSoup(url.text, "html.parser").prettify()                                               
                start = parse.find("<table")
                end = parse.find("</table")
                temp = pd.read_html(parse[start:end+8])[0]

                # --- Manipulate Data --- #
                temp.insert(0, 'Type', t)
                temp.insert(0, 'Month', month)
                temp.insert(0, 'Year', year)
                temp.columns = ['Year', 'Month', 'Type', 'From', 'To', 'Date', 'Player', 'Position', 'Transaction']

                # --- Combine Data --- #
                transactions = pd.concat([transactions, temp])
            except:
                print('No Data for: ' + t + ', ' + str(year) + ', ' + str(month))


            print(t + ', ' + str(year) + ', ' + str(month))

transactions.reset_index(drop = True, inplace = True)
transactions.to_csv('C:\\Users\\pensh\\Desktop\\VSCode\\DataBase\\Data\\Transactions\\Transactions.csv')
transactions

In [ ]:
### ----- Transactions1 ----- ###
warnings.filterwarnings("ignore")
Transactions1 = pd.DataFrame(columns=['Date', 'Team', 'Acquired', 'Relinquished', 'Notes'])
team = [#'49ers', 'Bears', 'Bengals', 'Bills', 'Broncos', 'Browns', 'Buccaneers', 'Cardinals', 'Chargers', 'Chiefs', 'Colts', 'Commanders', 'Cowboys', 'Dolphins', 'Eagles', 'Falcons', 'Giants', 'Jaguars',
    # 'Jets', 'Lions', 'Packers', 'Panthers', 'Patriots', 'Raiders',
     'Rams', 
     'Ravens', 'Saints', 'Seahawks', 'Steelers', 'Texans', 'Titans', 'Vikings']

for t in team:
    for r in range(0, 16111, 1):
        page = r * 25
        # --- Collect Data --- #
        s = 'https://www.prosportstransactions.com/football/Search/SearchResults.php?Player=&Team=%s&BeginDate=&EndDate=&PlayerMovementChkBx=yes&ILChkBx=yes&NBADLChkBx=yes&InjuriesChkBx=yes&PersonalChkBx=yes&DisciplinaryChkBx=yes&LegalChkBx=yes&submit=Search&start=%s' % (t, str(page))
        url = requests.get(s)
        print(url)
        parse = BeautifulSoup(url.text, "html.parser").prettify()                                               
        start = parse.find("<table")
        end = parse.find("</table")
        temp = pd.read_html(parse[start:end+8])[0]

        # --- Manipulate Data --- #
        temp.columns = ['Date', 'Team', 'Acquired', 'Relinquished', 'Notes']
        temp = temp[temp['Date'] != 'Date']

        # --- Break Check --- #
        if len(temp.index) == 0:
            break

        # --- Combine Data --- #
        Transactions1 = pd.concat([Transactions1, temp])

        print('Team: ' + t + ', page ' + str(r))

    endLink = 'C:\\Users\\pensh\\Desktop\\VSCode\\DataBase\\Data\\Transactions\\%s.csv' % t
    Transactions1.to_csv(endLink)
Transactions1

In [ ]:
# --- Collect Data --- #
page = 0
s = 'https://www.prosportstransactions.com/football/Search/SearchResults.php?Player=&Team=49ers&BeginDate=&EndDate=&PlayerMovementChkBx=yes&ILChkBx=yes&NBADLChkBx=yes&InjuriesChkBx=yes&PersonalChkBx=yes&DisciplinaryChkBx=yes&LegalChkBx=yes&submit=Search&start=%s' % str(2500)
url = requests.get(s)
print(url)
parse = BeautifulSoup(url.text, "html.parser").prettify()                                               
start = parse.find("<table")
end = parse.find("</table")
temp = pd.read_html(parse[start:end+8])[0]

# --- Manipulate Data --- #
# temp.insert(0, 'Year', year)
temp.columns = ['Date', 'Team', 'Acquired', 'Relinquished', 'Notes']

temp

In [56]:
### --- Clean Data --- ###

team = ['49ers', 'Bears', 'Bengals', 'Bills', 'Broncos', 'Browns', 'Buccaneers', 'Cardinals', 'Chargers', 'Chiefs', 'Colts', 'Commanders', 'Cowboys', 'Dolphins', 'Eagles', 'Falcons', 'Giants', 'Jaguars', 'Jets', 'Lions', 'Packers', 'Panthers', 'Patriots', 'Raiders', 'Rams', 'Ravens', 'Saints', 'Seahawks', 'Steelers', 'Texans', 'Titans', 'Vikings']
df = pd.DataFrame()
for t in team:
    temp = pd.read_csv('C:\\Users\\pensh\\Desktop\\VSCode\\DataBase\\Data\\Transactions\\%s.csv' % t)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    temp['Acquired'] = temp['Acquired'].str.replace('• ', '')
    temp['Relinquished'] = temp['Relinquished'].str.replace('• ', '')
    df = pd.concat([df, temp])

# df[df['Acquired'].str.contains("Le'Veon Bell") == True]

conditions = [df['Team'].str.contains('49ers'),
              df['Team'] == 'Bears',
              df['Team'].str.contains('Bengals'),
              df['Team'].str.contains('Bills'),
              df['Team'].str.contains('Broncos'),
              df['Team'].str.contains('Browns'),
              df['Team'] == 'Buccaneers',
              df['Team'] == 'Cardinals',
              df['Team'].str.contains('Chargers'),
              (df['Team'].str.contains('Chiefs')) | (df['Team'] == 'Texans (AFL)'),
              df['Team'] == 'Colts',
              (df['Team'] == 'Commanders') | (df['Team'] == 'Washington') | (df['Team'] == 'Redskins'),
              df['Team'] == 'Cowboys',
              df['Team'].str.contains('Dolphins'),
              df['Team'] == 'Eagles',
              df['Team'] == 'Falcons',
              df['Team'] == 'Giants',
              df['Team'] == 'Jaguars',
              (df['Team'].str.contains('Jets')) | (df['Team'] == 'Titans (AFL)'),
              df['Team'] == 'Lions',
              (df['Team'] == 'Minneapolis (AFL)') | (df['Team'] == 'Vikings'),
              (df['Team'].str.contains('Oilers')) | (df['Team'] == 'Titans'),
              df['Team'] == 'Packers',
              df['Team'] == 'Panthers',
              df['Team'].str.contains('Patriots'),
              (df['Team'] == 'Pirates') | (df['Team'] == 'Steelers'),
              df['Team'].str.contains('Raiders'),
              df['Team'] == 'Rams',
              df['Team'] == 'Ravens',
              df['Team'] == 'Saints',
              df['Team'] == 'Seahawks',
              df['Team'] == 'Texans',
              df['Team'] == 'Wranglers (USFL)',
              df['Team'] == 'Gamblers (USFL)',
              df['Team'] == 'Breakers (USFL)',
              df['Team'] == 'Bandits (USFL)',]
franchises = ['49ers', 'Bears', 'Bengals', 'Bills', 'Broncos', 'Browns', 'Buccaneers', 'Cardinals', 'Chargers', 'Chiefs', 'Colts', 'Commanders', 'Cowboys', 'Dolphins', 'Eagles', 'Falcons', 'Giants', 'Jaguars', 'Jets', 'Lions', 'Vikings', 'Titans', 'Packers', 'Panthers', 'Patriots', 'Steelers', 'Raiders', 'Rams', 'Ravens', 'Saints', 'Seahawks', 'Texans', 'Wranglers', 'Gamblers', 'Breakers', 'Bandits']

df['Franchise'] = np.select(conditions, franchises, "UNKNOWN")

def league(t):
    if '(AFL)' in t:
        return 'AFL'
    elif '(AAFC)' in t:
        return 'AAFC'
    elif '(USFL)' in t:
        return 'USFL'
    elif '(' not in t:
        return 'NFL'
    else:
        return 'UNKNOWN'

df['League'] = df['Team'].apply(league)
df['League'].unique()

array(['AAFC', 'NFL', 'AFL', 'USFL'], dtype=object)

In [3]:
team = ['49ers', 'Bears', 'Bengals', 'Bills', 'Broncos', 'Browns', 'Buccaneers', 'Cardinals', 'Chargers', 'Chiefs', 'Colts', 'Commanders', 'Cowboys', 'Dolphins', 'Eagles', 'Falcons', 'Giants', 'Jaguars', 'Jets', 'Lions', 'Packers', 'Panthers', 'Patriots', 'Raiders', 'Rams', 'Ravens', 'Saints', 'Seahawks', 'Steelers', 'Texans', 'Titans', 'Vikings']
df = pd.DataFrame()
for t in team:
    temp = pd.read_csv('C:\\Users\\pensh\\Desktop\\VSCode\\DataBase\\Data\\Transactions\\%s.csv' % t)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    temp['Acquired'] = temp['Acquired'].str.replace('• ', '')
    temp['Relinquished'] = temp['Relinquished'].str.replace('• ', '')
    df = pd.concat([df, temp])

df.drop_duplicates(inplace = True)
df.reset_index(drop = True, inplace = True)

df.loc[((df['Date'] == '1899-12-30') & (df['Acquired'] == 'Ahmad Merritt')), 'Date'] = '2000-04-16'
df.loc[((df['Date'] == '1899-12-30') & (df['Acquired'] == 'Chris Lammons')), 'Date'] = '2023-01-24'
df = df[df['Date'].str.contains('1899') == False]

# df[df['Team'] == 'Steelers']

In [23]:
df['Type'] = np.nan
df.loc[df['Notes'].str.split(' ')[0] == 'Signed', 'Type'] = 'Signed'
# temp = df['Notes'].str.split(' ').tolist()
# l = [print(item) for item in temp]

df

KeyError: 'cannot use a single bool to index into setitem'